In [1]:
import pandas as pd;
import datetime as dt
import time
import numpy as np
import bisect

In [27]:
dfHist = pd.read_csv('itemhistory.csv', dtype={'Craft':str, 'DateTime':dt.datetime, 'DefIndex':np.float64, 'Name':str,
                                            'PriceIndex':str, 'QualInt':np.int64, 'Trade':str, 'currency':str,
                                            'timestamp':np.float64, 'value':np.float64, 'value_high':np.float64})
dfHist['DateTime'] = pd.to_datetime(dfHist['DateTime'])
dfHist = dfHist.set_index(['Name', 'DefIndex', 'QualInt', 'Trade', 'Craft', 'PriceIndex', 'timestamp'])
dfHist.sort_index(inplace = True)

In [28]:
#Gets the history values for keys (currency in metal)
dfKey = dfHist.copy(deep = True).loc['Mann Co. Supply Crate Key']

In [29]:
#Gets the history values for metal (currency in USD)
dfMetal = dfHist.copy(deep = True).loc['Refined Metal'].loc[5002].loc[6]

In [30]:
#dates is a sorted list of DateTimes, pivot is a DateTime
#that you want to find the nearest previous date in dates
def prevDate(dates, pivot):
    insertNum = bisect.bisect_left(dates, pivot)
    if (insertNum != len(dates)):
#     if (insertNum != dates.size):
        if (dates[insertNum] == pivot):
            return insertNum
    if(insertNum == 0):
        return 0
    return (insertNum - 1)

In [31]:
#dates is a sorted list of DateTimes, pivot is a DateTime
#that you want to find the nearest previous date in dates
def nextDate(dates, pivot):
    insertNum = bisect.bisect_right(dates, pivot)
    if insertNum > 0:
        if (dates[insertNum - 1] == pivot):
            return insertNum - 1
        if (insertNum == len(dates)):
            return insertNum - 1
    return (insertNum)

In [32]:
dfMetalVal = dfMetal.reset_index().loc[:,'DateTime':]
dfMetalVal.drop('currency', inplace = True, axis = 1)
dfMetalVal['value'] = np.round((dfMetalVal['value'] + dfMetalVal['value_high'])/2, decimals = 2)
# dfMetalVal['value'] = (dfMetalVal['value'] + dfMetalVal['value_high'])/2
dfMetalVal.drop('value_high', inplace = True, axis = 1)

In [33]:
dfKeyVal = dfKey.copy(deep = True)

In [34]:
for ind, ser in dfKeyVal.iterrows():
    dfKeyVal.loc[ind, 'value'] = np.float64.round(((dfMetalVal.iloc[prevDate(dfMetalVal['DateTime'], ser['DateTime'])]['value'] +
                       dfMetalVal.iloc[nextDate(dfMetalVal['DateTime'], ser['DateTime'])]['value'])/2) * ser['value'], decimals = 2)
    dfKeyVal.loc[ind, 'value_high'] = np.float64.round(((dfMetalVal.iloc[prevDate(dfMetalVal['DateTime'], ser['DateTime'])]['value'] +
                       dfMetalVal.iloc[nextDate(dfMetalVal['DateTime'], ser['DateTime'])]['value'])/2) * ser['value_high'], decimals = 2)

In [35]:
dfKeyVal = dfKeyVal.reset_index().loc[:,'DateTime':]
dfKeyVal.drop('currency', inplace = True, axis = 1)
dfMetalVal['value'] = np.round(dfMetalVal['value'], decimals = 2)
# dfKeyVal['value'] = (dfKeyVal['value'] + dfKeyVal['value_high'])/2
dfKeyVal.drop('value_high', inplace = True, axis = 1)

In [38]:
for ind, ser in dfHist.iterrows():
    if (ser['currency'] == 'metal'):
        dfHist.loc[ind, 'value'] = np.float64.round(((dfMetalVal.iloc[prevDate(dfMetalVal['DateTime'], ser['DateTime'])]['value'] +
                           dfMetalVal.iloc[nextDate(dfMetalVal['DateTime'], ser['DateTime'])]['value'])/2) * ser['value'], decimals = 2)
        dfHist.loc[ind, 'value_high'] = np.float64.round(((dfMetalVal.iloc[prevDate(dfMetalVal['DateTime'], ser['DateTime'])]['value'] +
                           dfMetalVal.iloc[nextDate(dfMetalVal['DateTime'], ser['DateTime'])]['value'])/2) * ser['value_high'], decimals = 2)
        dfHist.loc[ind, 'currency'] = 'usd'
    elif (ser['currency'] == 'keys'):
        dfHist.loc[ind, 'value'] = np.float64.round(((dfKeyVal.iloc[prevDate(dfKeyVal['DateTime'], ser['DateTime'])]['value'] +
                           dfKeyVal.iloc[nextDate(dfKeyVal['DateTime'], ser['DateTime'])]['value'])/2) * ser['value'], decimals = 2)
        dfHist.loc[ind, 'value_high'] = np.float64.round(((dfKeyVal.iloc[prevDate(dfKeyVal['DateTime'], ser['DateTime'])]['value'] +
                           dfKeyVal.iloc[nextDate(dfKeyVal['DateTime'], ser['DateTime'])]['value'])/2) * ser['value_high'], decimals = 2)
        dfHist.loc[ind, 'currency'] = 'usd'

In [41]:
dfDrop = dfHist.copy(deep = True)

In [43]:
dfDrop = dfDrop[dfDrop['currency'] == 'usd']

In [46]:
#This date is the first time metal was found. Before that USD Prices are off.
#So we drop everything before that date.
date = pd.to_datetime("2013-01-20 00:49:02")
delta = pd.Timedelta('0 days')

In [48]:
dfDrop = dfDrop[dfDrop['DateTime'] - date >= delta]

In [52]:
dfDrop.to_csv('itemhistoryUSD.csv')